In [ ]:
import folium
from folium.features import GeoJsonTooltip
import fiona
import geopandas
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
import pandas as pd
from folium.plugins import HeatMap

import pandas as pd
import geopandas as gpd

import geopandas.tools
#from geopy.geocoders import Nominatim
#import geopy
from geopandas.tools import geocode
#from geopy.extra.rate_limiter import RateLimiter
from difflib import SequenceMatcher


import numpy as np

from shapely.geometry import Point, Polygon

In [ ]:
zoning = gpd.read_file("shp_files/Zoning_Regulations_of_2016.geojson")

In [ ]:
zoning.head(2)

,ZONING,ZONING_WEB_URL,ZONING_CHANGE_NARRATIVE,ZONING_STATUS,ZONING_LABEL,ZONE_DESCRIPTION,ZONE_DISTRICT,IZ_DESIGNATION,ZR58,ZR16,GLOBALID,OBJECTID,SHAPEAREA,SHAPELEN,geometry
0,MU-7B,https://handbook.dcoz.dc.gov/zones/mixed-use/M...,"Fixed map error of SSL 4507 936, 07/14/11, Mis...",Final,MU-7B,Permits medium density mixed-use development,Mixed-Use Zone,None,C-3-A,MU-7B,{CA941084-283B-4ADB-BDB4-2F5F4D5F8C33},515010,0,0,"POLYGON ((-76.98059 38.90251, -76.98070 38.902..."
1,MU-4,https://handbook.dcoz.dc.gov/zones/mixed-use/M...,"Official Digital Zoning Map of July 1, 2010",Final,MU-4,Permits moderate density mixed use development,Mixed-Use Zone,None,C-2-A,MU-4,{EE555AF3-9EAA-4FBB-997A-CA12025CBCD6},515011,0,0,"POLYGON ((-76.99314 38.90401, -76.99304 38.903..."


In [ ]:
zoning['high-density'] = np.zeros((len(zoning), 1))

zoning['high-density'] = zoning['ZONE_DESCRIPTION'].apply(lambda x: 'yes' if x!= None and "high-density" in x else "no")


In [ ]:
zoning = zoning[zoning['high-density']=='yes']

In [ ]:
zoning.head(2)

,ZONING,ZONING_WEB_URL,ZONING_CHANGE_NARRATIVE,ZONING_STATUS,ZONING_LABEL,ZONE_DESCRIPTION,ZONE_DISTRICT,IZ_DESIGNATION,ZR58,ZR16,GLOBALID,OBJECTID,SHAPEAREA,SHAPELEN,geometry,high-density
8,D-5,https://handbook.dcoz.dc.gov/zones/downtown/d-5/,"Official Digital Zoning Map of July 1, 2010",Final,D-5,Promotes high-density development of commercia...,Downtown Zone,None,C-3-C,D-5,{82D40D04-5975-4E80-BADF-FAD083ABDF20},515018,0,0,"POLYGON ((-77.02808 38.90253, -77.02837 38.902...",yes
11,RA-5,https://handbook.dcoz.dc.gov/zones/residential...,"Official Digital Zoning Map of July 1, 2010",Final,RA-5,permits high-density apartments,Residential Apartment Zone,None,R-5-E,RA-5,{18788CFE-DB84-4DCD-AFDD-76FA136E0A25},515021,0,0,"POLYGON ((-77.04491 38.89736, -77.04501 38.897...",yes


In [ ]:
zoning.drop(['ZONING_WEB_URL', 'ZONING_CHANGE_NARRATIVE', 'ZONING_STATUS', 'ZONING_LABEL', 'ZONE_DESCRIPTION', 'IZ_DESIGNATION'], axis = 1, inplace = True)

In [ ]:
zoning.drop(['OBJECTID', 'SHAPEAREA'],axis = 1, inplace = True)

KeyError: "['OBJECTID', 'SHAPEAREA'] not found in axis"

In [ ]:
zoning['ZONE_DISTRICT'].unique()

array(['Downtown Zone', 'Residential Apartment Zone',
       'Special Purpose Zone', 'Mixed-Use Zone'], dtype=object)

In [ ]:
zoning_colour_dict = {'Downtown Zone': 0,
                    'Residential Apartment Zone': 1,
                    'Special Purpose Zone':2,
                    'Mixed-Use Zone':3}

In [ ]:
zoning['zone_district_colour'] = zoning['ZONE_DISTRICT'].map(zoning_colour_dict)

In [ ]:
zoning.head(2)

,ZONING,ZONE_DISTRICT,ZR58,ZR16,GLOBALID,SHAPELEN,geometry,high-density,zone_district_colour
8,D-5,Downtown Zone,C-3-C,D-5,{82D40D04-5975-4E80-BADF-FAD083ABDF20},0,"POLYGON ((-77.02808 38.90253, -77.02837 38.902...",yes,0
11,RA-5,Residential Apartment Zone,R-5-E,RA-5,{18788CFE-DB84-4DCD-AFDD-76FA136E0A25},0,"POLYGON ((-77.04491 38.89736, -77.04501 38.897...",yes,1


In [ ]:
zoning_just_df = zoning[['ZONING', 'zone_district_colour']]

In [ ]:
m = folium.Map(location=[38.900497,-77.007507],pointer_events=True, zoom_start = 13)
folium.Choropleth(geo_data= zoning, data = zoning_just_df, columns = ['ZONING', 'zone_district_colour'],
                    key_on='feature.properties.ZONING',
                    fill_color='YlOrRd', legend_name='zone type – downtown, residential, special purpose, mixed-use').add_to(m)
m